Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
import nltk
#seperating words
nltk.download('punkt_tab')
#removing insignificant words
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

Data Cleaning

In [45]:
#load data from csv
emotions_data = pd.read_csv("emotions.csv", encoding = "UTF-8")
emotions_data.head(5)

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [14]:
#shape of emotion data
emotions_data.shape

(416809, 2)

In [15]:
#check for missing values in dataset
emotions_data.isnull().sum()

text     0
label    0
dtype: int64

In [36]:
#check distribution of 'label' column
emotions_data['label'].value_counts()

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [48]:
#downsample
min_count = emotions_data['label'].value_counts().min()
selected_cols = ['label'] + [col for col in emotions_data.columns if col != 'label']
emotions_data = emotions_data.groupby('label')[selected_cols].apply(lambda x: x.sample(min_count, random_state=18)).reset_index(drop=True)
emotions_data['label'].value_counts()

label
0    14972
1    14972
2    14972
3    14972
4    14972
5    14972
Name: count, dtype: int64

In [49]:
emotions_data.head(10)

,label,text
0,0,i feel like the saddle got in the way or inhib...
1,0,i feel failure and discontent when they are no...
2,0,i am sticky to you sending me cutesy futsy mes...
3,0,i only feel its aching and sore
4,0,i feel that sometimes my lessons are too borin...
5,0,i feel kind of dull these days
6,0,i loved him with all my heart and i feel empty...
7,0,i feel like we broke up but skipping the actua...
8,0,i already feel ugly here on the mission and i ...
9,0,i am feeling rather stressed out fed up and ov...


Word2Vec Word Embedding

In [50]:
#loads pretrained model for vectorizing sentences, Word2Vec
path = "GoogleNews-vectors-negative300.bin.gz"
model = KeyedVectors.load_word2vec_format(path, binary=True)
vocab_size = len(model.index_to_key)

In [51]:
#method for mean pooling
tokenizer = nltk.RegexpTokenizer(r'\w+')
def embedSentence_meanPooling(row):
    #uses nltk to seperate sentence into words omitting punctuation
    tokens = tokenizer.tokenize(row)
    #averages all token vectors
    vector = np.array([0]*300)
    count = 0
    for i in tokens:
        if i in model: #only adds tokens present in the model
            vector = vector + model.get_vector(i)
            count += 1
    vector = vector/max(1, count)
    return vector

In [52]:
#embeds all x from word to values
emotions_data["text"] = emotions_data["text"].apply(embedSentence_meanPooling)

In [73]:
#set input and output
X = emotions_data["text"].values
y = emotions_data["label"].values

In [74]:
#compress X into batch dimension vector to feed into tensorflow
X = np.stack(X).astype(np.float32)
X.shape

(89832, 300)

In [55]:
#split x, y into training
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=0)

In [56]:
#builds tensorflow model
tf_model = tf.keras.Sequential([
                                tf.keras.layers.Dense(12, activation='relu'),
                                tf.keras.layers.Dense(6, activation='softmax')
                                ])

In [57]:
tf_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=["accuracy"])

In [62]:
tf_model.evaluate(X_train, y_train)

1685/1685 ━━━━━━━━━━━━━━━━━━━━ 1s 673us/step - accuracy: 0.7131 - loss: 0.7674


[0.7664663195610046, 0.7137423753738403]

In [61]:
tf_model.fit(X_train, y_train, batch_size=16, epochs = 10, validation_data=(X_valid, y_valid))

Epoch 1/10
3369/3369 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6915 - loss: 0.8421 - val_accuracy: 0.6931 - val_loss: 0.8432
Epoch 2/10
3369/3369 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.6935 - loss: 0.8322 - val_accuracy: 0.6957 - val_loss: 0.8348
Epoch 3/10
3369/3369 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6977 - loss: 0.8184 - val_accuracy: 0.6963 - val_loss: 0.8286
Epoch 4/10
3369/3369 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6997 - loss: 0.8125 - val_accuracy: 0.6966 - val_loss: 0.8208
Epoch 5/10
3369/3369 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7010 - loss: 0.8056 - val_accuracy: 0.6972 - val_loss: 0.8160
Epoch 6/10
3369/3369 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7044 - loss: 0.8021 - val_accuracy: 0.6956 - val_loss: 0.8149
Epoch 7/10
3369/3369 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.7030 - loss: 0.7950 - val_accuracy: 0.6908 - val_loss: 0.8242
Epoch 8/10
3369/3369 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.7079 - loss: 0.7863 - 

In [81]:
#test on one sample sentence
sample_sentence = np.stack(embedSentence_meanPooling("god i would happy dying right now")).astype(np.float32)
sample_sentence = np.expand_dims(sample_sentence, axis=0)   
predicted = np.argmax(tf_model.predict(sample_sentence), axis=1)
predicted

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


array([1], dtype=int64)